Use in VS Code
Create a python environment. I used 3.12.2
activate the kernel for the env- I called mine jupy 

In [ ]:
# Cell 1: Use ! to run shell (wsl/linux I guess?) commands from an IPython notebook cell, windows seems to want %
%pip install numpy pandas gradio matplotlib pyarrow ipywidgets 


In [ ]:
# Cell 2 Import necessary libraries
import numpy as np
import pandas as pd
import gradio as gr
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from mpl_toolkits.mplot3d import Axes3D
import os

# Ensure the output directory exists
output_dir = 'fractal_outputs'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Cell 3: Fractal Generation Functions
def plot_fractal(fractal_type, depth_or_iter, c_real=0.0, c_imag=0.0, filename_prefix="fractal"):
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.axis('off')


def sierpinski_triangle(vertices, depth):
    """Generate Sierpinski triangle fractal images.

    Args:
        ax (matplotlib.axes.Axes): Axes to draw the fractal on.
        order (int): Order of recursion.
        vertices (array): Array of x,y coordinates for triangle vertices.
        filename_prefix (str): Prefix for saved image filenames.
    """

    if depth == 0:
        return [vertices]
    else:
        # Find midpoints of the triangle's edges
        mid_01 = (vertices[0] + vertices[1]) / 2
        mid_12 = (vertices[1] + vertices[2]) / 2
        mid_20 = (vertices[2] + vertices[0]) / 2
        # Recursively generate smaller triangles
        triangle1 = sierpinski_triangle([vertices[0], mid_01, mid_20], depth - 1)
        triangle2 = sierpinski_triangle([mid_01, vertices[1], mid_12], depth - 1)
        triangle3 = sierpinski_triangle([mid_20, mid_12, vertices[2]], depth - 1)
        return triangle1 + triangle2 + triangle3

def mandelbrot(c, max_iter):
    z = 0
    n = 0
    while abs(z) <= 2 and n < max_iter:
        z = z*z + c
        n += 1
    return n

def julia_set(c, max_iter, x_min, x_max, y_min, y_max, width, height):
    julia = np.zeros((height, width))
    for x in range(width):
        for y in range(height):
            zx = x * (x_max - x_min) / (width - 1) + x_min
            zy = y * (y_max - y_min) / (height - 1) + y_min
            z = complex(zx, zy)
            n = 0
            while np.abs(z) <= 2 and n < max_iter:
                z = z * z + c
                n += 1
            julia[y, x] = n
    return julia




In [ ]:
# ipynb Cell 4: Gradio Interface Functions
def plot_fractal(fractal_type, depth_or_iter, c_real=0.0, c_imag=0.0):
    if fractal_type == "Sierpinski Triangle":
        # Generate triangle vertices
        vertices = np.array([[0, 0], [0.5, np.sqrt(3) / 2], [1, 0]])
        # Generate triangles
        triangles = sierpinski_triangle(vertices, depth_or_iter)
        fig, ax = plt.subplots()
        ax.set_aspect('equal')
        ax.axis('off')
        for triangle in triangles:
            ax.fill(*zip(*triangle), "b")
        filename = os.path.join(output_dir, 'sierpinski_triangle.png')
        plt.savefig(filename)
        plt.close()
        return filename
    elif fractal_type == "Julia Set":
        c = complex(c_real, c_imag)
        x_min, x_max, y_min, y_max = -1.5, 1.5, -1.5, 1.5
        width, height = 800, 800
        julia_image = julia_set(c, depth_or_iter, x_min, x_max, y_min, y_max, width, height)
        plt.imshow(julia_image, cmap='hot', extent=(x_min, x_max, y_min, y_max))
        plt.colorbar()
        plt.axis('off')
        filename = os.path.join(output_dir, 'julia_set.png')
        plt.savefig(filename)
        plt.close()
        return filename
    elif fractal_type == "Mandelbrot Set":
        x_min, x_max = -2, 1
        y_min, y_max = -1.5, 1.5
        width, height = 800, 800
        mandelbrot_image = np.zeros((height, width))
        for x in range(width):
            for y in range(height):
                real = x_min + x * (x_max - x_min) / width
                imag = y_min + y * (y_max - y_min) / height
                color = mandelbrot(complex(real, imag), depth_or_iter)
                mandelbrot_image[y, x] = color
        plt.imshow(mandelbrot_image, cmap='hot', extent=(x_min, x_max, y_min, y_max))
        plt.colorbar()
        plt.axis('off')
        filename = os.path.join(output_dir, 'mandelbrot_set.png')
        plt.savefig(filename)
        plt.close()
        return filename

In [9]:
# Cell 5: Gradio Interface Setup
iface = gr.Interface(
    fn=plot_fractal,
    inputs=[
        gr.Dropdown(choices=["Julia Set", "Mandelbrot Set", "Sierpinski Triangle"], label="Fractal Type"),
        gr.Slider(minimum=1, value=5, maximum=1000, label="Iterations: Past 10 on Sierpinski is too much for most PC"),
        gr.Number(label="Real part of c", value=-0.8),
        gr.Number(label="Imaginary part of c", value=0.156)
    ],
    outputs="image",
    title="Fractal Generator",
    description="Select a fractal type and adjust the parameters to generate the fractal."
)

iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
